# DS-SF-34 | 16 | Trees | Codealong | Answer Key

In [1]:
import os

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import math

## Part A - The 2008 Democratic Primaries

(dataset adapted from http://www.stat.ucla.edu/~cocteau/primaries.csv)

In [2]:
df = pd.read_csv(os.path.join('..', 'datasets', 'dataset-16-2008-democrat-primaries.csv'))

In [3]:
df.columns

Index([u'fips', u'county_name', u'state_postal', u'region', u'election_date',
       u'racetype', u'tvotes', u'clinton', u'obama', u'edwards', u'margin',
       u'winner', u'POP05_SQMI', u'popUnder30_00', u'pop65up_00',
       u'presVote04', u'kerry04', u'Bush04', u'pres04margin', u'pres04winner',
       u'pop06', u'pop00', u'hisp06', u'white06', u'black06', u'indian06',
       u'asian06', u'hawaii06', u'mixed06', u'pct_less_30k', u'pct_more_100k',
       u'pct_hs_grad', u'pct_labor_force', u'pct_homeowner', u'unempFeb07',
       u'unempFeb08', u'unempChg', u'pctUnins00', u'subForPctHomes',
       u'poverty05', u'median_hhi05', u'Catholic', u'So.Bapt.Conv',
       u'Un.Methodist', u'E.L.C.A.', u'Construction', u'Manufacturing',
       u'FinancialActivities', u'GoodsProducing', u'ServiceProviding'],
      dtype='object')

In [4]:
df['c'] = (df.winner == 'obama')

### First cut: Is a county more than 20% black?

In [5]:
df['pct_black06'] = df.black06 / df.pop06 # Fraction of black voters

In [6]:
parent_df = df
left_child_df = parent_df[parent_df.pct_black06 <= .2]
right_child_df = parent_df.drop(left_child_df.index)
# or "parent_df[parent_df.pct_black06 > .2]"

#### First cut/right node

In [7]:
(right_child_df.c == 1).sum() # Obama

381

In [8]:
(right_child_df.c == 0).sum() # Clinton

70

Obama wins these counties 381 to 70.  (383 to 70 in The Obama-Clinton Divide decision tree)

In [9]:
def obama_vs_clinton(df):
    obama = (df.c == 1).sum()
    clinton = (df.c == 0).sum()
    if obama > clinton:
        print 'Obama wins these counties {} to {}.'.format(obama, clinton)
    elif clinton > obama:
        print 'Clinton wins these counties {} to {}.'.format(clinton, obama)
    else:
        print 'Obama and Clinton tie in these counties {} {}.'.format(obama, clinton)

In [10]:
obama_vs_clinton(right_child_df)

Obama wins these counties 381 to 70.


### Second cut: Is high school graduation rate higher than 78%?

In [11]:
parent_df = left_child_df
left_child_df = parent_df[parent_df.pct_hs_grad <= .78]
right_child_df = df.drop(left_child_df.index)
# or "parent_df[parent_df.pct_hs_grad > .78]"

In [12]:
obama_vs_clinton(left_child_df)

Clinton wins these counties 714 to 93.


Clinton wins these counties 704 to 89 in The Obama-Clinton Divide decison tree.

### Third cut: Is high school graduation rate higher than 87%?

In [13]:
parent_df = right_child_df
left_child_df = parent_df[parent_df.pct_hs_grad <= .87]
right_child_df = parent_df.drop(left_child_df.index)
# or "parent_df[parent_df.pct_hs_grad > .87]"

In [14]:
obama_vs_clinton(right_child_df)

Obama wins these counties 183 to 36.


Obama wins these counties 185 to 36 in The Obama-Clinton Divide decison tree.

## Part B - Building the 2008 Democratic Primaries Decision Tree by Hand

In [15]:
class Node:

    @staticmethod
    def root(root_df):
        cs = sorted(set(root_df.c))
        return Node(cs, root_df)

    def decision(self, left_filter):
        # Collect the observations for which the decision split is true and
        # create the corresponding left node

        left_filter = left_filter(self.df)
        left_df = self.df[left_filter]
        self.left = Node(self.cs, left_df)

        # Same thing on the right side but for the observations that don't
        # satisfy the decision split (the "else")

        right_df = self.df.drop(left_df.index)
        self.right = Node(self.cs, right_df)

        # The entropy after the decision split is the weighted average of the
        # children entropy

        self.after = (self.left.samples * self.left.before
                      + self.right.samples * self.right.before) / self.samples

        # The information gain corresponds to the entropy lost between the
        # parent node (this node and the "before") and its child (the "after")

        self.information_gain = self.before - self.after

        return self

    def __init__(self, cs, df):
        self.cs = cs
        self.df = df

        # Counts of the remaining observations in the subspace per classes
        self.counts = [(self.df.c == c).sum() for c in self.cs]

        # Number of observations in the subspace
        self.samples = sum(self.counts)

        # For empty subspaces, probabilties and entropy are set to zero
        if self.samples == 0:
            self.probabilities = [.0 for count in self.counts]
            self.before = .0
        else:
            self.probabilities = [1. * count / self.samples for count in self.counts]
            self.before = - sum(map(lambda p: p * math.log(p, 2),
                                    filter(lambda p : p > .0, self.probabilities)))

    def status(self):
        print 'classes                       =', self.cs
        print 'before:'
        print "\tparent:"
        print "\t\tsamples       =", self.samples
        print "\t\tcounts        =", self.counts
        print "\t\tprobabilities =", self.probabilities
        print "\t\tentropy       =", self.before
        print 'after:'
        print "\tleft child:"
        print "\t\tsamples       =", self.left.samples
        print "\t\tcounts        =", self.left.counts
        print "\t\tprobabilities =", self.left.probabilities
        print "\t\tentropy       =", self.left.before
        print "\tright child:"
        print "\t\tsamples       =", self.right.samples
        print "\t\tcounts        =", self.right.counts
        print "\t\tprobabilities =", self.right.probabilities
        print "\t\tentropy       =", self.right.before
        print
        print 'before entropy                =', self.before
        print 'after entropy                 =', self.after
        print 'information gain              =', self.information_gain

In [16]:
df.c = df.winner

### First cut

In [17]:
node = Node.root(df)

#### Candidate #1: Is a county more than 20% black?

In [18]:
node.decision(lambda df: df.pct_black06 <= .2).status()

classes                       = [nan, 'clinton', 'obama']
before:
	parent:
		samples       = 2241
		counts        = [0, 1210, 1031]
		probabilities = [0.0, 0.53993752788933513, 0.46006247211066487]
		entropy       = 0.995392878882
after:
	left child:
		samples       = 1791
		counts        = [0, 1141, 650]
		probabilities = [0.0, 0.63707426018983804, 0.36292573981016191]
		entropy       = 0.945085004347
	right child:
		samples       = 450
		counts        = [0, 69, 381]
		probabilities = [0.0, 0.15333333333333332, 0.84666666666666668]
		entropy       = 0.6181194891

before entropy                = 0.995392878882
after entropy                 = 0.879429278394
information gain              = 0.115963600488


#### Candidate #2: Is high school graduation rate higher than 78%?

In [19]:
node.decision(lambda df: df.pct_hs_grad <= .78).status()

classes                       = [nan, 'clinton', 'obama']
before:
	parent:
		samples       = 2241
		counts        = [0, 1210, 1031]
		probabilities = [0.0, 0.53993752788933513, 0.46006247211066487]
		entropy       = 0.995392878882
after:
	left child:
		samples       = 1167
		counts        = [0, 774, 393]
		probabilities = [0.0, 0.66323907455012854, 0.33676092544987146]
		entropy       = 0.92168535501
	right child:
		samples       = 1074
		counts        = [0, 436, 638]
		probabilities = [0.0, 0.4059590316573557, 0.5940409683426443]
		entropy       = 0.974329848577

before entropy                = 0.995392878882
after entropy                 = 0.946915246171
information gain              = 0.0484776327111


#### Candidate #3: Is high school graduation rate higher than 87%?

In [20]:
node.decision(lambda df: df.pct_hs_grad <= .87).status()

classes                       = [nan, 'clinton', 'obama']
before:
	parent:
		samples       = 2241
		counts        = [0, 1210, 1031]
		probabilities = [0.0, 0.53993752788933513, 0.46006247211066487]
		entropy       = 0.995392878882
after:
	left child:
		samples       = 2024
		counts        = [0, 1176, 848]
		probabilities = [0.0, 0.5810276679841897, 0.4189723320158103]
		entropy       = 0.980972219459
	right child:
		samples       = 217
		counts        = [0, 34, 183]
		probabilities = [0.0, 0.15668202764976957, 0.84331797235023043]
		entropy       = 0.62631249047

before entropy                = 0.995392878882
after entropy                 = 0.94662988961
information gain              = 0.0487629892719


> First cut: Selecting candidate #1: Is a county more than 20% black?

In [21]:
node = node.decision(lambda df: df.pct_black06 <= .2)

### Second cut

In [22]:
node = node.left # "left" is "county less than 20% black"

#### Candidate #1: Is a county more than 20% black?

In [23]:
node.decision(lambda df: df.pct_black06 <= .2).status()

classes                       = [nan, 'clinton', 'obama']
before:
	parent:
		samples       = 1791
		counts        = [0, 1141, 650]
		probabilities = [0.0, 0.63707426018983804, 0.36292573981016191]
		entropy       = 0.945085004347
after:
	left child:
		samples       = 1791
		counts        = [0, 1141, 650]
		probabilities = [0.0, 0.63707426018983804, 0.36292573981016191]
		entropy       = 0.945085004347
	right child:
		samples       = 0
		counts        = [0, 0, 0]
		probabilities = [0.0, 0.0, 0.0]
		entropy       = 0.0

before entropy                = 0.945085004347
after entropy                 = 0.945085004347
information gain              = 0.0


> Again, no information gain since this candidate was already selected for the first cut.

#### Candidate #2: Is high school graduation rate higher than 78%?

In [24]:
node.decision(lambda df: df.pct_hs_grad <= .78).status()

classes                       = [nan, 'clinton', 'obama']
before:
	parent:
		samples       = 1791
		counts        = [0, 1141, 650]
		probabilities = [0.0, 0.63707426018983804, 0.36292573981016191]
		entropy       = 0.945085004347
after:
	left child:
		samples       = 801
		counts        = [0, 708, 93]
		probabilities = [0.0, 0.88389513108614237, 0.11610486891385768]
		entropy       = 0.518059807076
	right child:
		samples       = 990
		counts        = [0, 433, 557]
		probabilities = [0.0, 0.43737373737373736, 0.56262626262626259]
		entropy       = 0.988653582507

before entropy                = 0.945085004347
after entropy                 = 0.778187019626
information gain              = 0.166897984722


#### Candidate #3: Is high school graduation rate higher than 87%?

In [25]:
node.decision(lambda df: df.pct_hs_grad <= .87).status()

classes                       = [nan, 'clinton', 'obama']
before:
	parent:
		samples       = 1791
		counts        = [0, 1141, 650]
		probabilities = [0.0, 0.63707426018983804, 0.36292573981016191]
		entropy       = 0.945085004347
after:
	left child:
		samples       = 1580
		counts        = [0, 1107, 473]
		probabilities = [0.0, 0.70063291139240502, 0.29936708860759492]
		entropy       = 0.880515856611
	right child:
		samples       = 211
		counts        = [0, 34, 177]
		probabilities = [0.0, 0.16113744075829384, 0.83886255924170616]
		entropy       = 0.637023743365

before entropy                = 0.945085004347
after entropy                 = 0.851829739417
information gain              = 0.0932552649308


> Second cut: Selecting candidate #2: Is high school graduation rate higher than 78%?

In [26]:
node = node.decision(lambda df: df.pct_hs_grad <= .78)

### Third cut

In [27]:
node = node.right # "right" is "high school graduation rate higher than 78%"

#### Candidate #1: Is a county more than 20% black?

In [28]:
node.decision(lambda df: df.pct_black06 <= .2).status()

classes                       = [nan, 'clinton', 'obama']
before:
	parent:
		samples       = 990
		counts        = [0, 433, 557]
		probabilities = [0.0, 0.43737373737373736, 0.56262626262626259]
		entropy       = 0.988653582507
after:
	left child:
		samples       = 990
		counts        = [0, 433, 557]
		probabilities = [0.0, 0.43737373737373736, 0.56262626262626259]
		entropy       = 0.988653582507
	right child:
		samples       = 0
		counts        = [0, 0, 0]
		probabilities = [0.0, 0.0, 0.0]
		entropy       = 0.0

before entropy                = 0.988653582507
after entropy                 = 0.988653582507
information gain              = 0.0


> Again, no information gain since this candidate was already selected for the first cut.

#### Candidate #2: Is high school graduation rate higher than 78%?

In [29]:
node.decision(lambda df: df.pct_hs_grad <= .78).status()

classes                       = [nan, 'clinton', 'obama']
before:
	parent:
		samples       = 990
		counts        = [0, 433, 557]
		probabilities = [0.0, 0.43737373737373736, 0.56262626262626259]
		entropy       = 0.988653582507
after:
	left child:
		samples       = 0
		counts        = [0, 0, 0]
		probabilities = [0.0, 0.0, 0.0]
		entropy       = 0.0
	right child:
		samples       = 990
		counts        = [0, 433, 557]
		probabilities = [0.0, 0.43737373737373736, 0.56262626262626259]
		entropy       = 0.988653582507

before entropy                = 0.988653582507
after entropy                 = 0.988653582507
information gain              = 0.0


> No information gain since this candidate was already selected for the second cut.

#### Candidate #3: Is high school graduation rate higher than 87%?

In [30]:
node.decision(lambda df: df.pct_hs_grad <= .87).status()

classes                       = [nan, 'clinton', 'obama']
before:
	parent:
		samples       = 990
		counts        = [0, 433, 557]
		probabilities = [0.0, 0.43737373737373736, 0.56262626262626259]
		entropy       = 0.988653582507
after:
	left child:
		samples       = 779
		counts        = [0, 399, 380]
		probabilities = [0.0, 0.51219512195121952, 0.48780487804878048]
		entropy       = 0.999570839347
	right child:
		samples       = 211
		counts        = [0, 34, 177]
		probabilities = [0.0, 0.16113744075829384, 0.83886255924170616]
		entropy       = 0.637023743365

before entropy                = 0.988653582507
after entropy                 = 0.922300700709
information gain              = 0.0663528817986


> Third cut: Selecting candidate #3: Is high school graduation rate higher than 87%?